In [95]:
%matplotlib inline

import numpy as np
import pandas as pd
import os, sys

import matplotlib.pyplot as plt
import seaborn as sns

import sqlite3

from collections import Counter, defaultdict
from bs4 import BeautifulSoup

from numpy import random

import warnings
warnings.filterwarnings('ignore')

np.random.seed(1)

basepath = os.path.expanduser('~/Desktop/src/paul-the-octopus/')
sys.path.append(os.path.join(basepath, 'src'))

In [69]:
with sqlite3.connect(os.path.join(basepath,'data/raw/database.sqlite')) as con:
    countries = pd.read_sql_query("SELECT * from Country", con)
    matches = pd.read_sql_query("SELECT * from Match", con, parse_dates=['date'])
    leagues = pd.read_sql_query("SELECT * from League", con)
    teams = pd.read_sql_query("SELECT * from Team", con)
    players = pd.read_sql_query("SELECT * from Player", con)
    players_stats = pd.read_sql_query("SELECT * from Player_Stats", con)
    

* Countries table contain information regarding country id and country name
* Matches contain information about a league game which has information related to fixture,
  and how the two teams were set up, match statistics and details about events that took place.
* Leagues contain league id, country id and name of the league
* Teams contain information about team id, team api id and team long and short name
* Players contain information about a player e.g. player api id, player name, fifa id, birthday and height, weight
* Players_stats contains player statistics obtained from fifa

In [70]:
# consider only english premier league for now.
english_league_id = countries.loc[countries.name == 'England', 'id'].values[0]

** League Id and Country Id for English Premier League is same for now. **

In [114]:
epl = matches.loc[matches.country_id == english_league_id]

In [76]:
list(epl.columns[77:85])

['goal',
 'shoton',
 'shotoff',
 'foulcommit',
 'card',
 'cross',
 'corner',
 'possession']

In [112]:
get_match_statistics(epl.iloc[:1], 'card')

team_id: 10260 and home_team_id: 10260
<class 'str'>
<class 'numpy.int64'>


team_id: 10260 and home_team_id: 10260
<class 'str'>
<class 'numpy.int64'>


team_id: 10260 and home_team_id: 10260
<class 'str'>
<class 'numpy.int64'>




(array([ 0.]), array([ 3.]))

In [107]:
BeautifulSoup(epl.possession.iloc[1])

<html><body><possession><value><comment>65</comment><event_incident_typefk>352</event_incident_typefk><elapsed>27</elapsed><subtype>possession</subtype><sortorder>0</sortorder><awaypos>35</awaypos><homepos>65</homepos><n>67</n><type>special</type><id>375608</id></value><value><comment>61</comment><elapsed_plus>2</elapsed_plus><event_incident_typefk>352</event_incident_typefk><elapsed>45</elapsed><subtype>possession</subtype><sortorder>1</sortorder><awaypos>39</awaypos><homepos>61</homepos><n>114</n><type>special</type><id>375663</id></value><value><comment>65</comment><event_incident_typefk>352</event_incident_typefk><elapsed>74</elapsed><subtype>possession</subtype><sortorder>3</sortorder><awaypos>35</awaypos><homepos>65</homepos><n>177</n><type>special</type><id>375787</id></value><value><comment>66</comment><elapsed_plus>3</elapsed_plus><event_incident_typefk>352</event_incident_typefk><elapsed>90</elapsed><subtype>possession</subtype><sortorder>2</sortorder><awaypos>34</awaypos><ho

### Features

In [113]:
def get_formation(match_api_id, team_type):
    formation_dict = Counter([matches.loc[matches.match_api_id == match_api_id]['%s_player_Y%d'%(team_type, i)].values[0] for i in range(1, 12)])
    sorted_keys = sorted(formation_dict)

    formation = ''
    for key in sorted_keys[1:-1]:
        y = formation_dict[key]
        formation += '%d-' % y
    formation += '%d' % formation_dict[sorted_keys[-1]]
    
    return formation

def decompose_season(matches):
    """
    Convert to ['2008/2009', '2009/2010'] into ([2008, 2009], [2009, 2010])
    """
    season_start = []
    season_end = []
    
    for season in matches.season:
        season_split = season.split('/')
        season_start.append(int(season_split[0]))
        season_end.append(int(season_split[1]))
    
    return (season_start, season_end)

def calculate_result(match_df):
    """
    Calculate results based on number of goals scored by home and away teams.
    """
    home_team_goals = match_df['home_team_goal']
    away_team_goals = match_df['away_team_goal']
    
    if home_team_goals > away_team_goals:
        return 'home'
    elif away_team_goals > home_team_goals:
        return 'away'
    else:
        return 'draw'

def get_match_statistics(match_df, feature):
    home_team_id = match_df['home_team_api_id']
    away_team_id = match_df['away_team_api_id']
        
    statistics = match_df[feature]

    home_team_counts = np.zeros(len(statistics))
    away_team_counts = np.zeros(len(statistics))

    for i, stat in enumerate(statistics):
        if stat is not None:
            parsed_xml = BeautifulSoup(stat, 'xml')

            for v in parsed_xml.find_all('value'):
                if v.find('team') is not None:
                    team_id = v.find('team').text
                    
                    if team_id == str(home_team_id.iloc[i]):
                        home_team_counts[i] += 1
                    else:
                        away_team_counts[i] += 1

    return (home_team_counts, away_team_counts)

In [115]:
epl['home_team_formation'] = epl.match_api_id.apply(get_formation, args=('home', ))
epl['away_team_formation'] = epl.match_api_id.apply(get_formation, args=('away',))

epl['season_start'], epl['season_end'] = decompose_season(epl)
epl['result'] = epl.apply(calculate_result, axis=1)

epl['home_team_cards'], epl['away_team_cards'] = get_match_statistics(epl, 'card')
epl['home_team_shots_on_target'], epl['away_team_shots_on_target'] = get_match_statistics(epl, 'shoton')
epl['home_team_shots_off_target'], epl['away_team_shots_off_target'] = get_match_statistics(epl, 'shotoff')
epl['home_team_crosses'], epl['away_team_crosses'] = get_match_statistics(epl, 'cross')
epl['home_team_corners'], epl['away_team_corners'] = get_match_statistics(epl, 'corner')
epl['home_team_fouls'], epl['away_team_fouls'] = get_match_statistics(epl, 'foulcommit')

In [116]:
features = ['home_team_api_id', 'away_team_api_id','season', 'stage', 'date', \
            'home_team_goal', 'away_team_goal', 'home_team_formation', 'away_team_formation',\
            'season_start', 'season_end', 'result', 'home_team_cards', 'away_team_cards',\
            'home_team_shots_on_target', 'away_team_shots_on_target', 'home_team_shots_off_target',\
            'away_team_shots_off_target', 'home_team_corners', 'away_team_corners', 'home_team_crosses',\
            'away_team_crosses', 'home_team_fouls', 'away_team_fouls'
           ]
            
processed = epl[features]

In [117]:
processed.head()

,home_team_api_id,away_team_api_id,season,stage,date,home_team_goal,away_team_goal,home_team_formation,away_team_formation,season_start,...,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_corners,away_team_corners,home_team_crosses,away_team_crosses,home_team_fouls,away_team_fouls
1728,10260,10261,2008/2009,1,2008-08-17,1,1,4-4-2,4-4-1-1,2008,...,11.0,1.0,10.0,9.0,6.0,6.0,24.0,9.0,16.0,11.0
1729,9825,8659,2008/2009,1,2008-08-16,1,0,4-4-2,4-5-1,2008,...,12.0,2.0,13.0,3.0,7.0,5.0,21.0,7.0,11.0,9.0
1730,8472,8650,2008/2009,1,2008-08-16,0,1,4-4-2,4-4-2,2008,...,4.0,11.0,3.0,5.0,1.0,8.0,15.0,19.0,13.0,12.0
1731,8654,8528,2008/2009,1,2008-08-16,2,1,4-4-2,4-4-2,2008,...,5.0,7.0,7.0,15.0,6.0,10.0,15.0,27.0,14.0,13.0
1732,10252,8456,2008/2009,1,2008-08-17,4,2,4-4-2,4-5-1,2008,...,5.0,9.0,4.0,5.0,7.0,8.0,16.0,16.0,11.0,13.0
